In [2]:
%pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

df_accountdim = pd.read_csv("/Users/hetapatel/Datathon 3-28-25/consumer_spending_ml/forecast_data/account_dim_20250325.csv")

In [4]:
df_accountdim.head()

,current_account_nbr,client_id,open_date,card_activation_date,card_activation_flag,ebill_ind,date_in_collection,overlimit_type_flag,payment_hist_1_12_mths,payment_hist_13_24_mths,employee_code,external_status_reason_code,special_finance_charge_ind,pscc_ind,ext_status_reason_cd_desc,account_card_type
0,nTgalbpd5Rsol7CA,YCTHAA278DC,2020-07-17T00:00:00.000-04:00,NaN,8,E,NaN,0,ZZZZZZIAZZZZ,ZZZZIIIQIIII,NaN,0,NaN,0,NaN,DUAL CARD
1,xYaLrD9IvmbVl1PB,CCSP489PLCC,2024-07-24T00:00:00.000-04:00,2024-08-09T00:00:00.000-04:00,0,E,NaN,0,IQIIIQIA,"\\\\\\""""",NaN,0,NaN,1,NaN,DUAL CARD
2,JM78WNziiRWgmL8e,HGBHARRFT984,2024-05-05T00:00:00.000-04:00,2024-05-05T00:00:00.000-04:00,0,"\\\\\\""""",NaN,0,IIIQIQIIIIA,"\\\\\\""""",NaN,0,NaN,0,NaN,PLCC
3,bv4NLVhWTGCf8KgC,OSELW263,2024-07-28T00:00:00.000-04:00,2024-09-08T00:00:00.000-04:00,0,"\\\\\\""""",NaN,0,I1III1A,"\\\\\\""""",NaN,0,NaN,0,NaN,PLCC
4,nrXceQabYIHo31lp,HGBHARRFT984,2024-09-15T00:00:00.000-04:00,2024-09-15T00:00:00.000-04:00,0,"\\\\\\""""",NaN,0,"\\\\\\""""","\\\\\\""""",NaN,0,NaN,0,NaN,PLCC
